In [1]:
import torch
import torch.nn.functional as F

In [3]:
falstaff = open("input.txt").read()
print(len(falstaff))

1115394


In [11]:
# Create Datasets
chars = set("".join(falstaff))
num_embeddings = len(chars)
num_embeddings

65

In [14]:
# Create Mapping Dicts
itos = {i:s for i,s in enumerate(chars)}
stoi = {s:i for i,s in itos.items()}

In [16]:
encode = lambda input: [stoi[char] for char in input]
decode = lambda input: [itos[token] for token in input]

#test:
enc = (encode("doth mother know that you wereth 'er drapes?"))
dec = (decode(enc))

print(enc)
print(dec)

[62, 6, 5, 48, 44, 13, 6, 5, 48, 46, 0, 44, 49, 4, 6, 45, 44, 5, 48, 12, 5, 44, 21, 6, 43, 44, 45, 46, 0, 46, 5, 48, 44, 47, 46, 0, 44, 62, 0, 12, 63, 46, 27, 10]
['d', 'o', 't', 'h', ' ', 'm', 'o', 't', 'h', 'e', 'r', ' ', 'k', 'n', 'o', 'w', ' ', 't', 'h', 'a', 't', ' ', 'y', 'o', 'u', ' ', 'w', 'e', 'r', 'e', 't', 'h', ' ', "'", 'e', 'r', ' ', 'd', 'r', 'a', 'p', 'e', 's', '?']


In [17]:
# Creating Datasets
data = torch.tensor(encode(falstaff))
data.shape

torch.Size([1115394])

In [20]:
# Test, Train splits
train = int(0.9 * data.shape[0])
train

1003854

In [22]:
training_data = data[:train]
training_data.shape

torch.Size([1003854])

In [24]:
testing_data = data[train:]
testing_data.shape

torch.Size([111540])

In [ ]:
# Data Loader:
# The idea is to have random samplings from the data and then creating batches from it.
batch_size = 4
context_len = 8

# This means that the transformer is going to process four "streams" of tokens, each upto 8 tokens long

# I need arbitary starting points for data
rand_idx = torch.randint(0,(training_data.shape[0] - (context_len)),(training_data.shape))
rand_idx


tensor([950895, 854916, 390166,  ..., 870704, 222753, 923272])

In [36]:
# Now, I need to batch these
# Actually, I can be smarter here:

def data_loader(data,batch_size,context_len):
    rand_idx = torch.randint(0,(data.shape[0] - context_len),(batch_size,))
    X = torch.stack([data[idx:idx+context_len] for idx in rand_idx])
    Y = torch.stack([data[idx+1:idx+context_len+1] for idx in rand_idx])
    return X,Y

In [37]:
data_loader(training_data,batch_size=4,context_len=8)

(tensor([[46,  0, 27, 44, 28,  6,  0, 44],
         [12, 27, 44,  1,  0, 43, 46, 31],
         [44,  5, 48, 17, 27, 44, 62, 17],
         [ 4, 44, 46, 17, 36, 48,  5,  3]]),
 tensor([[ 0, 27, 44, 28,  6,  0, 44,  5],
         [27, 44,  1,  0, 43, 46, 31, 44],
         [ 5, 48, 17, 27, 44, 62, 17, 50],
         [44, 46, 17, 36, 48,  5,  3, 21]]))